In [1]:
import os
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [2]:
# Laden der Umgebungsvariablen aus der .env-Datei
load_dotenv()
uri = os.environ['DB_URI']
# Herstellen der Verbindung zur MongoDB-Datenbank
client = MongoClient(uri)
db = client["rosen"]
collection = db["big_data"]

In [6]:
# Funktion zum Ersetzen von NaN-Werten durch den Mittelwert der benachbarten Werte
def replace_nan(values, attribute):
    new_values = []
    changed = False  # Flag, um zu überprüfen, ob Werte geändert wurden
    
    if attribute == 'Distance':
        # Durchführung einer linearen Regression
        x = np.arange(len(values))
        valid_indices = ~np.isnan(values)
        slope, intercept, _, _, _ = linregress(x[valid_indices], values[valid_indices])
        # Ersetzen von NaN-Werten durch den linearen Wert
        new_values = [slope * i + intercept if pd.isna(val) else val for i, val in enumerate(values)]
        changed = True
    elif attribute in ['magnetization', 'wall thickness']:
        # Überprüfen, ob NaN-Werte am Anfang oder am Ende vorhanden sind
        first_valid_index = next((i for i, val in enumerate(values) if not pd.isna(val)), None)
        last_valid_index = next((i for i, val in enumerate(values[::-1]) if not pd.isna(val)), None)
        last_valid_index = len(values) - last_valid_index - 1 if last_valid_index is not None else None
        
        if first_valid_index is not None and last_valid_index is not None and (last_valid_index - first_valid_index) >= 10:
            # Wenn genügend Werte vorhanden sind, wird der Durchschnitt der 5 vorherigen und 5 nachfolgenden Werte berechnet
            new_values = values.copy()
            for i, val in enumerate(values):
                if pd.isna(val):
                    start = max(0, i - 5)
                    end = min(len(values), i + 6)
                    valid_values = [v for v in values[start:end] if not pd.isna(v)]
                    if valid_values:
                        new_values[i] = np.nanmean(valid_values)
                        changed = True
                    else:
                        new_values[i] = np.nan
        else:
            # Wenn nicht genügend Werte vorhanden sind, wird der Durchschnitt der gesamten Messreihe verwendet
            mean_value = np.nanmean(values)
            new_values = [mean_value if pd.isna(val) else val for val in values]
            changed = True
    else:
        # Wenn das Attribut nicht 'Distance', 'magnetization' oder 'wall thickness' ist, bleibt der Wert unverändert
        new_values = values
        
    return new_values, changed
# Daten abrufen
documents = collection.find({})
# Liste für gefundene IDs
found_ids = []
data_sets = []
# Iterieren über jedes Dokument
for document in documents:
    measuring_points = document["measuring_points"]
    has_nan = False
    for key, values in measuring_points.items():
        # Überprüfen, ob NaN-Werte in den Werten vorhanden sind
        if any(pd.isna(values)):
            has_nan = True
            # Ersetzen von NaN-Werten entsprechend dem Attribut
            new_values, changed = replace_nan(values, key)
            if changed:
                measuring_points[key] = new_values
    
    if has_nan:
        # Aktualisieren des Dokuments in der Datenbank
        # collection.update_one({"_id": document["_id"]}, {"$set": {"measuring_points": measuring_points}})
        # Hinzufügen der ID zum DataFrame
        found_ids.append(document['_id'])
        data_sets.append(measuring_points) 
       

# Einfaches Printen der ID 
#print(f"NaN-Werte gefunden in Messgröße '{key}' des Dokuments mit der ID '{document['_id']}'")
            

In [9]:
 # Erstellen des DataFrame
    df = pd.DataFrame({
    "_id": found_ids,
    "measuring_points": data_sets
})

# Ausgabe des DataFrames
print(df)
df[10:15]

                                      _id  \
0    007ff213-9b5d-4243-9c8b-5eae997ac0ee   
1    0108e474-f1f4-4274-bf9f-e51d4963c091   
2    023794f2-5e3d-4450-8b71-8aae79d381a1   
3    0245cbf0-faec-4708-a1f2-3ccf7ebec1e1   
4    02498907-b394-4b7c-a0ab-068692746591   
..                                    ...   
568  faecadfb-5753-411f-984d-e653753418f6   
569  fbd4d95f-a969-4ebc-b464-13952662bb36   
570  fbf0453e-50a3-4f6c-9415-b1477de29725   
571  fed1458a-14a9-41af-b32a-1fda98e285e6   
572  ff1de9c2-7098-4d51-9e75-1d5990fcec55   

                                      measuring_points  
0    {'defect_channel': [0.0, 1.0, 0.0, 0.0, 0.0, 1...  
1    {'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0...  
2    {'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0...  
3    {'defect_channel': [1.0, 0.0, 1.0, 0.0, 0.0, 0...  
4    {'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0...  
..                                                 ...  
568  {'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0...  
569

,_id,measuring_points
10,057cbcdd-e626-4703-8848-da770f70f92a,"{'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0..."
11,061aa681-4dbe-4d5d-8425-83404ae59405,"{'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0..."
12,0624c030-7ff7-44b7-a031-8b3f1f93ca60,"{'defect_channel': [1.0, 0.0, 0.0, 0.0, 0.0, 0..."
13,0625e935-7fca-4791-95e0-de7a74d48c96,"{'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0..."
14,0634770d-02ae-4abe-a24e-66b9c72ec6fe,"{'defect_channel': [0.0, 0.0, 0.0, 0.0, 0.0, 0..."


Bis hier wurden die NaN-Werte vollständig bereinigt und ersetzt. Dabei sind nur die attribute 'magnetization', 'wall thickness' und 'distance' geändert. Als nächstes werden fehlender Werte der  Velocity errechnet.

In [3]:
file = collection.find_one({"_id": "00f5bd09-dbc5-4434-9327-63c91ed75551"}, {"_id": 1, "measuring_points": 1})
#file = collection.find_one({"_id": "01c70e2e-144c-4719-8095-17d2fd2a5e3b"}, {"_id": 1, "measuring_points": 1})

In [8]:
df = pd.DataFrame(file.get("measuring_points"))
df[40:50]

,defect_channel,distance,magnetization,timestamp,velocity,wall_thickness
0,0.0,0.000000,-0.900778,1.375422e+09,0.494565,10.596800
1,0.0,0.459459,-0.926663,1.375423e+09,0.462401,11.115602
2,0.0,0.918919,-1.000078,1.375424e+09,0.411980,12.860558
3,0.0,1.378378,-0.870686,1.375425e+09,0.445213,10.806081
4,0.0,1.837838,-0.060898,1.375426e+09,0.448074,10.324141
5,0.0,2.297297,2.804358,1.375427e+09,0.534397,6.184383
6,0.0,2.756757,-0.196725,1.375428e+09,0.452160,10.474002
7,0.0,3.216216,0.217796,1.375429e+09,0.468961,11.209519
8,0.0,3.675676,2.982575,1.375430e+09,0.495490,5.500545
9,0.0,4.135135,2.872076,1.375431e+09,0.381207,5.346162
